Code to support the Project Presentation 

In [15]:
!pip install python-tsp >/dev/null
!pip install pulp >/dev/null
!pip install pyproj >/dev/null

In [4]:
from sklearn.metrics.pairwise import haversine_distances
from math import radians
from python_tsp.exact import solve_tsp_dynamic_programming
import pandas as pd

import numpy as np

from python_tsp.exact import solve_tsp_brute_force
from pulp import *
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import itertools
from scipy.spatial import distance_matrix
import matplotlib
import seaborn as sns
import io
import warnings
from google.colab import files
warnings.filterwarnings("ignore")

In [5]:
uploaded = files.upload()
xls = pd.ExcelFile(io.BytesIO(uploaded['city_location.xlsx']))

Saving city_location.xlsx to city_location.xlsx


In [6]:
df2 = pd.read_excel(xls, sheet_name='Sheet 2',skiprows = 1,header = 0,index_col=None,usecols = "B:C",names = ['City','Coordinates'])
df1 = pd.read_excel(xls, sheet_name='Sheet1',skiprows = 1,header = 0,index_col=None,usecols = "B:C",names = ['City','Coordinates'])
frames = [df1,df2]
df = pd.concat(frames,ignore_index=True)
df.describe()
len(df['City'].unique())
df=df.drop_duplicates(subset='City',ignore_index=True)
lat=[]#latitude
lon=[]#longtitude
for row in df['Coordinates']:
    lat.append(row.split(',')[0])
    lon.append(row.split(',')[1])
df['Latitude'] = lat
df['Longitude'] = lon
df = df.drop(columns = ['Coordinates'])
#set New York to be the depot
df = pd.concat([df.iloc[[4],:], df.drop(4, axis=0)], axis=0)
df = df.reset_index()
df = df.drop(columns = ['index'])
# df.to_excel('cityunique.xlsx')
# df.reindex([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18])

In [16]:
newdf1 = df
N = len(newdf1['City'])
newdf1;

In [22]:
flight_time = [[]]
distance = [[]]
for i in range(N):
    value = []
    valued = []
    for j in range(N):
        if(i != j):
            CityA = newdf1.loc[i]
            CityB = newdf1.loc[j]
            coordA = [float(CityA.Latitude),float(CityA.Longitude)]
            coordB = [float(CityB.Latitude),float(CityB.Longitude)]
            A_rad = [radians(_) for _ in coordA]
            B_rad = [radians(_) for _ in coordB]
            result = haversine_distances([A_rad, B_rad])
            value.append(result[0][1]*6371000/1000/926+1.5)
            valued.append(result[0][1]*6371000/1000)
        else: 
            value.append(int(0))
            valued.append(int(0))
    vessel = []
    vessel = [value[j] for j in range(N)]
    vessel2 = []
    vessel2 = [valued[j] for j in range(N)]   
    flight_time.append(vessel)
    distance.append(vessel2)
ft = pd.DataFrame(flight_time, index=None, columns = newdf1['City'])
ddis = pd.DataFrame(distance, index=None, columns = newdf1['City'])
ft=ft.drop([0])
ddis = ddis.drop([0])
distance_matrix=ddis.to_numpy()
time_matrix = ft.to_numpy()
# distance = ddis.set_index(newdf1['City'])
# flighttime = ft.set_index(newdf1['City'])

## Crashes if city exceeds 20

In [34]:
permutation, distance = solve_tsp_dynamic_programming(distance_matrix)

In [33]:
permutation, time = solve_tsp_dynamic_programming(time_matrix)

In [39]:
total_distance = distance + distance_matrix[permutation[-1]][[permutation[0]]]
total_time = time + time_matrix[permutation[-1]][[permutation[0]]]
Cities = [(newdf1.loc[i]['City']) for i in permutation]
Cities.append(Cities[0])

In [43]:
print(f'Flying time: {total_time[0]:.2f} hrs')
print(f'Total Distance: {total_distance[0]:.2f} kms')
print(f'Route:{Cities}')

Flying time: 88.46 hrs
Total Distance: 54136.26 kms
Route:['New York ', 'Toronto', 'Chicago', 'Los Angeles', 'Seattle', 'Tokyo', 'Seoul', 'Shanghai ', 'Taipei', 'Sydney', 'Singapore', 'Doha', 'Istanbul ', 'Vienna', 'Milan', 'Barcelona', 'Madrid', 'Paris', 'London', 'New York ']


The rest is appendix: 